In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
# create a data generator
datagen = ImageDataGenerator(
    rescale=1.0/255,
    horizontal_flip = True,
    validation_split=0.1)

# load and iterate training dataset
train_it = datagen.flow_from_directory(
    '../master-data/train_small/rgb', 
    target_size=(224, 224),
    subset="training", 
    class_mode="categorical",
    shuffle=True,
    batch_size=32)

val_it = datagen.flow_from_directory(
    '../master-data/train_small/rgb', 
    target_size=(224, 224), 
    subset="validation", 
    class_mode="categorical",
    shuffle=True,
    batch_size=32)

Found 27011 images belonging to 11 classes.
Found 3000 images belonging to 11 classes.


In [3]:
def plot_history(history):
    '''Plot loss and accuracy as a function of the epoch,
    for the training and validation datasets.
    '''
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    # Get number of epochs
    epochs = range(len(acc))

    # Plot training and validation accuracy per epoch
    plt.plot(epochs, acc, label="acc")
    plt.plot(epochs, val_acc, label="val_acc")
    plt.title('Training and validation accuracy')
    plt.legend()
    
    # Plot training and validation loss per epoch
    plt.figure()

    plt.plot(epochs, loss, label="loss")
    plt.plot(epochs, val_loss, label="val_loss")
    plt.title('Training and validation loss')
    plt.legend()
    
    plt.show()

In [4]:
conv_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

2021-11-21 23:34:35.962370: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-11-21 23:34:36.043437: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:1a:00.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2021-11-21 23:34:36.044254: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 1 with properties: 
pciBusID: 0000:1d:00.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2021-11-21 23:34:36.056848: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-11-21 23:34:36.097061: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.

In [5]:
for layer in conv_model.layers:
    layer.trainable = False

# flatten the output of the convolutional part: 
x = Flatten()(conv_model.output)
# three hidden layers
x = Dense(100, activation='relu')(x)
x = Dense(100, activation='relu')(x)
x = Dense(100, activation='relu')(x)
# final softmax layer with two categories (dog and cat)
predictions = Dense(11, activation='softmax')(x)

# creating the full model:
full_model = Model(inputs=conv_model.input, outputs=predictions)

In [6]:
full_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
from tensorflow.keras.optimizers import Adam

full_model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=0.0001),
                  metrics=['acc'])

history = full_model.fit_generator(
    train_it, 
    validation_data = val_it,
    workers=14,
    epochs=10,
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10


2021-11-21 23:34:43.310562: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-11-21 23:34:44.872079: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7


845/845 [==============================] - 138s 164ms/step - loss: 2.3755 - acc: 0.1624 - val_loss: 2.3615 - val_acc: 0.1667
Epoch 2/10
453/845 [===============>..............] - ETA: 54s - loss: 2.3586 - acc: 0.1735

In [ ]:
plot_history(history)

In [ ]:
full_model.save_weights('resnet50.h5')

In [ ]:
conv_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
for layer in conv_model.layers:
    layer.trainable = False

# flatten the output of the convolutional part: 
x = Flatten()(conv_model.output)
# three hidden layers
x = Dense(100, activation='relu')(x)
x = Dense(100, activation='relu')(x)
x = Dense(100, activation='relu')(x)
# final softmax layer with two categories (dog and cat)
predictions = Dense(11, activation='softmax')(x)

# creating the full model:
full_model = Model(inputs=conv_model.input, outputs=predictions)
full_model.load_weights('resnet50.h5')


from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input

img_path = '../master-data/train_small/rgb/AcrimSat/image_00003_img.png'
img = image.load_img(img_path, target_size=(224,224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
predict = full_model.predict(x)
print(predict)
plt.imshow(img)